In [344]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 
import copy

Populating the interactive namespace from numpy and matplotlib


In [345]:
class Node:
    def __init__(self,feature_ind,klasslabel=0,score=0,split=0,fidx=-1):
        self.lchild=None
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.feature_ind=feature_ind
        
    def set_childs(self,lchild,rchild):
        self.lchild=lchild
        self.rchild=rchild

    def navigate(self, point):
        if(not self.isleaf()):
            if(point[self.feature_ind] > self.split):
                return self.rchild.navigate(point)
            else:
                return self.lchild.navigate(point)
        
        return self.klasslabel
            
    def isleaf(self):
        if self.lchild == None and self.rchild == None:
            return True
        return False

    def isless_than_eq(self, X):
        if X<=fidx:
            return True
        return False
        
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.feature_ind)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split) 

In [346]:
## Your code goes here...

class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
    def __init__(self, purityp, exthreshold,maxdepth=5,tree=None):        
        self.purity=purityp
        self.exthreshold=exthreshold
        self.maxdepth=maxdepth
        self.tree=tree
        
    def train(self, X, Y):
        
        nexamples,nfeatures=X.shape
        self.root = self.build_tree(X, Y, self.maxdepth)
        
    def get_optimal_split(self, X, Y):
        #Provide Features
        #Returns Right and Left Child Data and Labels
        #Returns Best Possible Split Node
        
        max_info_gain = -1
        split_val = 0
        r_child = []
        l_child = []
        label = -1
        
        nexamples, nfeatures=X.shape
    
        for i in range(nfeatures): #Choose Feature With Max Info Gain
            split, info_gain, rci, lci = self.evaluate_numerical_attribute(X[:,i], Y)
            if(info_gain > max_info_gain):
                max_info_gain = info_gain
                split_val = split
                r_child = rci
                l_child = lci
                label = i
        
        R_X = X[r_child]
        L_X = X[l_child]
        R_Y = Y[r_child]
        L_Y = Y[l_child]
        
        (values,counts) = np.unique(Y,return_counts=True)
        ind=np.argmax(counts)
        maj_lbl = values[ind]
    
        node = Node(label, maj_lbl, max_info_gain, split_val)

        return node, L_X, R_X, L_Y, R_Y
        
    def build_tree(self, X, Y, depth):

        nexamples, nfeatures=X.shape
        klasses,C=np.unique(Y,return_counts=True);
        
        if(len(X) == 1):
            node = Node(0, Y[0], 0, 0)
            return node
        
        elif(len(X) == 0):
            return None
        
        root, lx, rx, ly, ry = self.get_optimal_split(X, Y)
        
        if(depth > 0):
            root.set_childs(self.build_tree(lx, ly, depth-1),self.build_tree(rx, ry, depth-1))
        
        return root
        
    def test(self, X):
        
        nexamples, nfeatures=X.shape
        pclasses=self.predict(X)
        return pclasses
    
    def evaluate_numerical_attribute(self,feat, Y):
        
        classes=np.unique(Y)
        nclasses=len(classes)

        f = np.asarray(feat)
        
        # YOUR CODE HERE
        score = -99999
        size = len(Y)
        RCI = []
        LCI = []
        
        unique, counts = np.unique(f, return_counts=True)
        genie = 0
        for i in counts:
            genie += (i/size)**2
        genie = 1.0 - genie
        
        i = min(f)
        while i <= max(f):
            leftChildInd = []
            rightChildInd = []
            L_Ind = []
            R_Ind = []
            
            for j in range(0, len(f)):
                if f[j] <= i:
                    leftChildInd.append(Y[j])
                    L_Ind.append(j)
                else:
                    rightChildInd.append(Y[j])
                    R_Ind.append(j)
                    
            l_unique, l_counts = np.unique(leftChildInd, return_counts=True)                  
            r_unique, r_counts = np.unique(rightChildInd, return_counts=True)

            l_genie = 0
            len_left = len(leftChildInd)
            for x in l_counts:
                l_genie += (x/len_left)**2
            l_genie = 1.0-l_genie

            r_genie = 0
            len_right = len(rightChildInd)
            for x in r_counts:
                r_genie += (x/len_right)**2
            r_genie = 1.0-r_genie

            info_gain = genie-(((len_left/size))*l_genie)-(((len_right/size)*r_genie))

            if(info_gain > score):
                score = info_gain
                split = i
                RCI = np.asarray(R_Ind)
                LCI = np.asarray(L_Ind)
                    
            i += 0.01
                    
        return split, score, RCI, LCI
        
    def predict(self, X):
        
        pclass = []
        for i in X:
            pclass.append(self.root.navigate(i))

        return pclass
    
    def _predict(self,node, X):
        pass
      
    def __str__(self):
        return self.__print(self.tree) 
        
    #def find_depth(self):
        
        #return self._find_depth(self.tree)
    
    
    def _find_depth(self,node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild),self._find_depth(node.rchild))+1
        
    def __print(self,node,depth=0):
        
        ret = ""

        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild,depth+1)

        # Print own value
        
        ret += "\n" + ("    "*depth) + node.get_str()

        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild,depth+1)
        
        return ret

In [347]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

In [348]:
#load the data set
data=pd.read_csv('./iris.data')
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
#print (data.describe())

In [349]:
# Get your data in matrix (X ,Y)
test_ind = [1,5,13,18,27,31,46,54,68,72,80,87,90,96,101,111,116,123,137,142]
no_of_cols = len(data.columns)
X_data = np.asarray(data)
Y_data = X_data[:,no_of_cols-1:]

X = np.delete(Xd, test_ind, axis=0)
Y = X[:,no_of_cols-1:]
X = X[:,:no_of_cols-1]

print (" Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape)   

 Data Set Dimensions= (129, 4)  True Class labels dimensions (129, 1)


In [350]:
print (len(Y))
feat=[0,1]
dt=DecisionTree(0.95,5,2)
feat=[0,1]
dt.classes=np.unique(Y)
dt.nclasses=len(np.unique(Y))
split,mingain,Xlidx,Xridx=dt.evaluate_numerical_attribute(X[:,0],Y)
print("Split Value: ", split)
print("Information Gain: ", mingain)
# You should get following result:,
# Split=5.45, entropy=0.388707191825

129
Split Value:  5.409999999999976
Information Gain:  0.5108487070087934


In [351]:
# %pdb
#print (" Plotting the Decision Surface of Training Set... ")
#t.plot_decision_regions(X[:,feat],Y,clf=dt, res=0.1, cycle_marker=True, legend=1)

In [352]:
#*****************************Because SPLIT was not working!*****************************

Xtest = X_data[test_ind]
Ytest = Y_data[test_ind]

print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape)   

 Test Data Set Dimensions= (20, 5) Test True Class labels dimensions (20, 1)


# Let's See How Good we are doing

In [353]:
dt=DecisionTree(0.95,5)
dt.train(X,Y)

# Lets Train on All 4 Features and all 3 classes 

In [357]:
pclasses=dt.predict(Xtest)
score = 0
for i in range(len(pclasses)):
    if(Ytest[i] == pclasses[i]):
        score+=1
    else:
        print("Missclassified! :")
    print("Label:",Ytest[i]," Prediction:", pclasses[i])
score = score*100
print()
print("Accuracy: ", score/len(Ytest), "%")

Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-setosa']  Prediction: Iris-setosa
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-versicolor']  Prediction: Iris-versicolor
Label: ['Iris-virginica']  Prediction: Iris-virginica
Label: ['Iris-virginica']  Prediction: Iris-virginica
Label: ['Iris-virginica']  Prediction: Iris-virginica
Label: ['Iris-virginica']  Prediction: Iris-virginica
Missclassified! :
Label: ['Iris-virginica']  Prediction: